In [1]:
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.7 MB/s eta 0:00:00


In [ ]:
!pip install pyspark py4j

In [11]:
from pyspark.sql.functions import col

In [4]:
import psycopg2

host = "dpg-ck7g9u7q54js73famlu0-a.oregon-postgres.render.com"
port = 5432
database = "codema"
user = "codema_user"
password = "TL35jKeo53oDQkPDsw6c587xWAMSCmd6"


try:
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cursor = connection.cursor()

    #Validación
    print("Conexión exitosa a la base de datos")

    #Consulta
    cursor.execute("SELECT * FROM receipt")

    # Almacenar en variable
    records = cursor.fetchall()

    # Mostrar
    for record in records:
        print(record)

    connection.close()

except psycopg2.Error as error:
    print("Error al conectarse a la base de datos:", error)


Conexión exitosa a la base de datos
(12, 'daniel', '2023-01-10', Decimal('122.00'))
(9, 'neil', '2023-01-10', Decimal('130.00'))
(7, 'Fabricio B', '2023-1-3', Decimal('150.00'))
(13, 'Jose', '2023-1-12', Decimal('18993.00'))
(14, 'Fabricio', '2023-1-6', Decimal('2022.00'))
(8, 'giomar', '2023-01-10', Decimal('145.00'))
(15, 'Piero', '2023-1-10', Decimal('123.00'))
(3, 'piero', '2023-1-10', Decimal('122222.00'))
(10, 'kathy', '2023-01-10', Decimal('122.00'))
(16, 'Kathy', '2023-1-19', Decimal('90.00'))
(17, 'Jose', '2023-1-24', Decimal('90.00'))
(11, 'Piero', '2023-01-10', Decimal('122.00'))


In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.23.jar -P /content/


--2023-11-24 14:48:00--  https://jdbc.postgresql.org/download/postgresql-42.2.23.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1005522 (982K) [application/java-archive]
Saving to: ‘/content/postgresql-42.2.23.jar.1’

postgresql-42.2.23. 100%[===================>] 981.96K  6.23MB/s    in 0.2s    

2023-11-24 14:48:01 (6.23 MB/s) - ‘/content/postgresql-42.2.23.jar.1’ saved [1005522/1005522]



In [6]:
from pyspark.sql import SparkSession

#Sesión de Spark y Controlador PostgreSQL
spark = SparkSession.builder.appName("example").config("spark.driver.extraClassPath", "/content/postgresql-42.2.23.jar").getOrCreate()

# Conn de bd de render
host = "dpg-ck7g9u7q54js73famlu0-a.oregon-postgres.render.com"
port = 5432
database = "codema"
user = "codema_user"
password = "TL35jKeo53oDQkPDsw6c587xWAMSCmd6"

#conexión
url = f"jdbc:postgresql://{host}:{port}/{database}"
properties = {"user": user, "password": password, "driver": "org.postgresql.Driver"}



In [9]:
    # Leer por pyspark
    df = spark.read.jdbc(url=url, table="premier_league", properties=properties)



    # Muestra el DataFrame
    df.show()



+---+----------+-------------------+----------------+----------------+----+----+---+----+----+---+----------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+-----+----+-----+----+----+----+----+---+----+----+----+----+-----+----+-----+--------+--------+-----+-----+-------+-------+-------+-------+-----+-------+-------+----+----+------+------+------+------+------+------+------+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+---------+---------+------+------+--------+--------+--------+--------+-----+--------+--------+-----+-----+-------+-------+-------+-------+
|Div|      Date|               Time|        HomeTeam|        AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|   Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|B365H|B365D|B365A| BWH| BWD| BWA| IWH| IWD| IWA|  PSH| PSD|  PSA| WHH| WHD| WHA| VCH|VCD| VCA|MaxH|MaxD|MaxA| AvgH|AvgD| AvgA|B365>2.5|B365<2.5|P>2.5|P<2.5|Max>2.5|

In [ ]:
# Crear una nueva sesión de Spark
spark = SparkSession.builder.appName("example").config("spark.driver.extraClassPath", "/content/postgresql-42.2.23.jar").getOrCreate()

# DF
matches_table = df.select("Div", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR")

# Guardar en la base de datos
matches_table.write.jdbc(url=url, table="matches", mode="overwrite", properties=properties)



In [8]:
    # Leer la tabla
    matches_df = spark.read.jdbc(url=url, table="matches", properties=properties)

    matches_df.show()


+---+----------+----------------+----------------+----+----+---+
|Div|      Date|        HomeTeam|        AwayTeam|FTHG|FTAG|FTR|
+---+----------+----------------+----------------+----+----+---+
| E0|09/08/2019|       Liverpool|         Norwich|   4|   1|  H|
| E0|10/08/2019|        West Ham|        Man City|   0|   5|  A|
| E0|10/08/2019|     Bournemouth|Sheffield United|   1|   1|  D|
| E0|10/08/2019|         Burnley|     Southampton|   3|   0|  H|
| E0|10/08/2019|  Crystal Palace|         Everton|   0|   0|  D|
| E0|10/08/2019|         Watford|        Brighton|   0|   3|  A|
| E0|10/08/2019|       Tottenham|     Aston Villa|   3|   1|  H|
| E0|11/08/2019|       Leicester|          Wolves|   0|   0|  D|
| E0|11/08/2019|       Newcastle|         Arsenal|   0|   1|  A|
| E0|11/08/2019|      Man United|         Chelsea|   4|   0|  H|
| E0|17/08/2019|         Arsenal|         Burnley|   2|   1|  H|
| E0|17/08/2019|     Aston Villa|     Bournemouth|   1|   2|  A|
| E0|17/08/2019|        B

In [17]:
# Tu código actual para seleccionar las columnas
general_stats_table = df.select("Div", "Date", "HomeTeam", "AwayTeam", "HS", "AS", "HST", "AST", "HF", "AF")

# Cambiar nombres de columnas
nombres_columnas = {
    "Div": "Division",
    "Date": "Fecha",
    "HomeTeam": "EquipoLocal",
    "AwayTeam": "EquipoVisitante",
    "HS": "TirosLocal",
    "AS": "TirosVisitante",
    "HST": "TirosAlArcoLocal",
    "AST": "TirosAlArcoVisitante",
    "HF": "FaltasLocal",
    "AF": "FaltasVisitante"
}

# Aplicar alias a las columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas.items():
    general_stats_table = general_stats_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)

# Escribir en la base de datos
general_stats_table.write.jdbc(url=url, table="generalfc_stats", mode="overwrite", properties=properties)

In [18]:
    # Leer la tabla
    stats_df = spark.read.jdbc(url=url, table="generalfc_stats", properties=properties)

    stats_df.show()

+--------+----------+----------------+----------------+----------+--------------+----------------+--------------------+-----------+---------------+
|Division|     Fecha|     EquipoLocal| EquipoVisitante|TirosLocal|TirosVisitante|TirosAlArcoLocal|TirosAlArcoVisitante|FaltasLocal|FaltasVisitante|
+--------+----------+----------------+----------------+----------+--------------+----------------+--------------------+-----------+---------------+
|      E0|09/08/2019|       Liverpool|         Norwich|        15|            12|               7|                   5|          9|              9|
|      E0|10/08/2019|        West Ham|        Man City|         5|            14|               3|                   9|          6|             13|
|      E0|10/08/2019|     Bournemouth|Sheffield United|        13|             8|               3|                   3|         10|             19|
|      E0|10/08/2019|         Burnley|     Southampton|        10|            11|               4|              

In [19]:
# Tu código actual para seleccionar las columnas
cards_table = df.select("Div", "Date", "HomeTeam", "AwayTeam", "HY", "AY", "HR", "AR")

# Cambiar nombres de columnas
nombres_columnas_cards = {
    "Div": "Division",
    "Date": "Fecha",
    "HomeTeam": "EquipoLocal",
    "AwayTeam": "EquipoVisitante",
    "HY": "TarjetasAmarillasLocal",
    "AY": "TarjetasAmarillasVisitante",
    "HR": "TarjetasRojasLocal",
    "AR": "TarjetasRojasVisitante"
}

# Aplicar alias a las columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas_cards.items():
    cards_table = cards_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)

# Escribir en la base de datos
cards_table.write.jdbc(url=url, table="cards", mode="overwrite", properties=properties)

In [20]:
    # Leer la tabla
    cards_df = spark.read.jdbc(url=url, table="cards", properties=properties)

    cards_df.show()

+--------+----------+----------------+----------------+----------------------+--------------------------+------------------+----------------------+
|Division|     Fecha|     EquipoLocal| EquipoVisitante|TarjetasAmarillasLocal|TarjetasAmarillasVisitante|TarjetasRojasLocal|TarjetasRojasVisitante|
+--------+----------+----------------+----------------+----------------------+--------------------------+------------------+----------------------+
|      E0|09/08/2019|       Liverpool|         Norwich|                     0|                         2|                 0|                     0|
|      E0|10/08/2019|        West Ham|        Man City|                     2|                         2|                 0|                     0|
|      E0|10/08/2019|     Bournemouth|Sheffield United|                     2|                         1|                 0|                     0|
|      E0|10/08/2019|         Burnley|     Southampton|                     0|                         0|       

In [23]:
corners_table = df.select("Div", "Date", "HomeTeam", "AwayTeam", "HC", "AC")

# Cambiar nombres de columnas
nombres_columnas_corners = {
    "Div": "Division",
    "Date": "Fecha",
    "HomeTeam": "EquipoLocal",
    "AwayTeam": "EquipoVisitante",
    "HC": "CornersLocal",
    "AC": "CornersVisitante"
}

# Aplicar alias a las columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas_corners.items():
    corners_table = corners_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)


# Escribir en la base de datos
corners_table.write.jdbc(url=url, table="corners", mode="overwrite", properties=properties)

In [24]:
    # Leer la tabla
    corner_df = spark.read.jdbc(url=url, table="corners", properties=properties)

    corner_df.show()

+--------+----------+----------------+----------------+------------+----------------+
|Division|     Fecha|     EquipoLocal| EquipoVisitante|CornersLocal|CornersVisitante|
+--------+----------+----------------+----------------+------------+----------------+
|      E0|09/08/2019|       Liverpool|         Norwich|          11|               2|
|      E0|10/08/2019|        West Ham|        Man City|           1|               1|
|      E0|10/08/2019|     Bournemouth|Sheffield United|           3|               4|
|      E0|10/08/2019|         Burnley|     Southampton|           2|               7|
|      E0|10/08/2019|  Crystal Palace|         Everton|           6|               2|
|      E0|10/08/2019|         Watford|        Brighton|           5|               2|
|      E0|10/08/2019|       Tottenham|     Aston Villa|          14|               0|
|      E0|11/08/2019|       Leicester|          Wolves|          12|               3|
|      E0|11/08/2019|       Newcastle|         Arsenal

In [25]:
half_time_results_table = df.select("Div", "Date", "HomeTeam", "AwayTeam", "HTHG", "HTAG", "HTR")

# Cambiar nombres de columnas
nombres_columnas_half_time_results = {
    "Div": "Division",
    "Date": "Fecha",
    "HomeTeam": "EquipoLocal",
    "AwayTeam": "EquipoVisitante",
    "HTHG": "GolesMedioTiempoLocal",
    "HTAG": "GolesMedioTiempoVisitante",
    "HTR": "ResultadoMedioTiempo"
}

# Aplicar alias a las columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas_half_time_results.items():
    half_time_results_table = half_time_results_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)

# Escribir en la base de datos
half_time_results_table.write.jdbc(url=url, table="half_time_results", mode="overwrite", properties=properties)

In [26]:
    # Leer la tabla
    half_time_df = spark.read.jdbc(url=url, table="half_time_results", properties=properties)

    half_time_df.show()

+--------+----------+----------------+----------------+---------------------+-------------------------+--------------------+
|Division|     Fecha|     EquipoLocal| EquipoVisitante|GolesMedioTiempoLocal|GolesMedioTiempoVisitante|ResultadoMedioTiempo|
+--------+----------+----------------+----------------+---------------------+-------------------------+--------------------+
|      E0|09/08/2019|       Liverpool|         Norwich|                    4|                        0|                   H|
|      E0|10/08/2019|        West Ham|        Man City|                    0|                        1|                   A|
|      E0|10/08/2019|     Bournemouth|Sheffield United|                    0|                        0|                   D|
|      E0|10/08/2019|         Burnley|     Southampton|                    0|                        0|                   D|
|      E0|10/08/2019|  Crystal Palace|         Everton|                    0|                        0|                   D|


In [27]:
betting_odds_table = df.select("Div", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A")

# Cambiar nombres de columnas
nombres_columnas_betting_odds = {
    "Div": "Division",
    "Date": "Fecha",
    "HomeTeam": "EquipoLocal",
    "AwayTeam": "EquipoVisitante",
    "B365H": "CuotaVictoriaLocal",
    "B365D": "CuotaEmpate",
    "B365A": "CuotaVictoriaVisitante"
}

# Aplicar alias a las columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas_betting_odds.items():
    betting_odds_table = betting_odds_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)

# Escribir en la base de datos
betting_odds_table.write.jdbc(url=url, table="betting_odds", mode="overwrite", properties=properties)

In [28]:
    # Leer la tabla
    betting_odds_df = spark.read.jdbc(url=url, table="betting_odds", properties=properties)

    betting_odds_df.show()

+--------+----------+----------------+----------------+------------------+-----------+----------------------+
|Division|     Fecha|     EquipoLocal| EquipoVisitante|CuotaVictoriaLocal|CuotaEmpate|CuotaVictoriaVisitante|
+--------+----------+----------------+----------------+------------------+-----------+----------------------+
|      E0|09/08/2019|       Liverpool|         Norwich|              1.14|       10.0|                  19.0|
|      E0|10/08/2019|        West Ham|        Man City|              12.0|        6.5|                  1.22|
|      E0|10/08/2019|     Bournemouth|Sheffield United|              1.95|        3.6|                   3.6|
|      E0|10/08/2019|         Burnley|     Southampton|              2.62|        3.2|                  2.75|
|      E0|10/08/2019|  Crystal Palace|         Everton|               3.0|       3.25|                  2.37|
|      E0|10/08/2019|         Watford|        Brighton|               1.9|        3.4|                   4.0|
|      E0|

In [29]:
shots_on_target_table = df.select("Div", "Date", "HomeTeam", "AwayTeam", "HST", "AST")

# Cambiar nombres de columnas
nombres_columnas_shots_on_target = {
    "Div": "Division",
    "Date": "Fecha",
    "HomeTeam": "EquipoLocal",
    "AwayTeam": "EquipoVisitante",
    "HST": "TirosAlArcoLocal",
    "AST": "TirosAlArcoVisitante"
}

# Aplicar alias a las columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas_shots_on_target.items():
    shots_on_target_table = shots_on_target_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)

# Escribir en la base de datos
shots_on_target_table.write.jdbc(url=url, table="shots_on_target", mode="overwrite", properties=properties)

In [30]:
    # Leer la tabla
    shots_target_df = spark.read.jdbc(url=url, table="shots_on_target", properties=properties)

    shots_target_df.show()

+--------+----------+----------------+----------------+----------------+--------------------+
|Division|     Fecha|     EquipoLocal| EquipoVisitante|TirosAlArcoLocal|TirosAlArcoVisitante|
+--------+----------+----------------+----------------+----------------+--------------------+
|      E0|09/08/2019|       Liverpool|         Norwich|               7|                   5|
|      E0|10/08/2019|        West Ham|        Man City|               3|                   9|
|      E0|10/08/2019|     Bournemouth|Sheffield United|               3|                   3|
|      E0|10/08/2019|         Burnley|     Southampton|               4|                   3|
|      E0|10/08/2019|  Crystal Palace|         Everton|               2|                   3|
|      E0|10/08/2019|         Watford|        Brighton|               3|                   3|
|      E0|10/08/2019|       Tottenham|     Aston Villa|               7|                   4|
|      E0|11/08/2019|       Leicester|          Wolves|     

In [31]:
alternative_betting_odds_table = df.select(
    "Div", "Date", "HomeTeam", "AwayTeam", "BWH", "BWD", "BWA", "IWH", "IWD", "IWA", "WHH", "WHD", "WHA"
)

# Cambiar nombres de columnas
nombres_columnas_alternative_betting_odds = {
    "Div": "Division",
    "Date": "Fecha",
    "HomeTeam": "EquipoLocal",
    "AwayTeam": "EquipoVisitante",
    "BWH": "CuotaVictoriaLocalBetandWin",
    "BWD": "CuotaEmpateBetandWin",
    "BWA": "CuotaVictoriaVisitanteBetandWin",
    "IWH": "CuotaVictoriaLocalInterwetten",
    "IWD": "CuotaEmpateInterwetten",
    "IWA": "CuotaVictoriaVisitanteInterwetten",
    "WHH": "CuotaVictoriaLocalWilliamHill",
    "WHD": "CuotaEmpateWilliamHill",
    "WHA": "CuotaVictoriaVisitanteWilliamHill"
}

# Aplicar alias a las columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas_alternative_betting_odds.items():
    alternative_betting_odds_table = alternative_betting_odds_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)

# Escribir en la base de datos
alternative_betting_odds_table.write.jdbc(url=url, table="alternative_betting_odds", mode="overwrite", properties=properties)

In [32]:
    # Leer la tabla
    alternative_odds_df = spark.read.jdbc(url=url, table="alternative_betting_odds", properties=properties)

    alternative_odds_df.show()

+--------+----------+----------------+----------------+---------------------------+--------------------+-------------------------------+-----------------------------+----------------------+---------------------------------+-----------------------------+----------------------+---------------------------------+
|Division|     Fecha|     EquipoLocal| EquipoVisitante|CuotaVictoriaLocalBetandWin|CuotaEmpateBetandWin|CuotaVictoriaVisitanteBetandWin|CuotaVictoriaLocalInterwetten|CuotaEmpateInterwetten|CuotaVictoriaVisitanteInterwetten|CuotaVictoriaLocalWilliamHill|CuotaEmpateWilliamHill|CuotaVictoriaVisitanteWilliamHill|
+--------+----------+----------------+----------------+---------------------------+--------------------+-------------------------------+-----------------------------+----------------------+---------------------------------+-----------------------------+----------------------+---------------------------------+
|      E0|09/08/2019|       Liverpool|         Norwich|            

In [36]:
referee_stats_table = df.select("Div", "Date", "Referee", "HS", "AS", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR")

# Cambiar nombres de columnas
nombres_columnas_referee_stats = {
    "Div": "Division",
    "Date": "Fecha",
    "Referee": "Arbitro",
    "HS": "TirosLocal",
    "AS": "TirosVisitante",
    "HST": "TirosAlArcoLocal",
    "AST": "TirosAlArcoVisitante",
    "HF": "FaltasLocal",
    "AF": "FaltasVisitante",
    "HC": "CornesLocal",
    "AC": "CornesVisitante",
    "HY": "TarjetasAmarillasLocal",
    "AY": "TarjetasAmarillasVisitante",
    "HR": "TarjetasRojasLocal",
    "AR": "TarjetasRojasVisitante"
}

# Renombrar columnas
for antiguo_nombre, nuevo_nombre in nombres_columnas_referee_stats.items():
    referee_stats_table = referee_stats_table.withColumnRenamed(antiguo_nombre, nuevo_nombre)

# Escribir en la base de datos
referee_stats_table.write.jdbc(url=url, table="referee_fcstats", mode="overwrite", properties=properties)

In [37]:
    # Leer la tabla
    referee_df = spark.read.jdbc(url=url, table="referee_fcstats", properties=properties)

    referee_df.show()

+--------+----------+----------+----------+--------------+----------------+--------------------+-----------+---------------+-----------+---------------+----------------------+--------------------------+------------------+----------------------+
|Division|     Fecha|   Arbitro|TirosLocal|TirosVisitante|TirosAlArcoLocal|TirosAlArcoVisitante|FaltasLocal|FaltasVisitante|CornesLocal|CornesVisitante|TarjetasAmarillasLocal|TarjetasAmarillasVisitante|TarjetasRojasLocal|TarjetasRojasVisitante|
+--------+----------+----------+----------+--------------+----------------+--------------------+-----------+---------------+-----------+---------------+----------------------+--------------------------+------------------+----------------------+
|      E0|09/08/2019|  M Oliver|        15|            12|               7|                   5|          9|              9|         11|              2|                     0|                         2|                 0|                     0|
|      E0|10/08/2019

In [ ]:
    # Definir las tablas bidimensionales
    matches_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR")
    general_stats_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "HS", "AS", "HST", "AST", "HF", "AF")
    cards_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "HY", "AY", "HR", "AR")
    corners_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "HC", "AC")
    half_time_results_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "HTHG", "HTAG", "HTR")
    betting_odds_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A")
    shots_on_target_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "HST", "AST")
    substitutions_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam")  # Ajuste aquí
    alternative_betting_odds_table = premier_df.select("Div", "Date", "HomeTeam", "AwayTeam", "BWH", "BWD", "BWA", "IWH", "IWD", "IWA", "WHH", "WHD", "WHA")

    # Escribir las tablas en la base de datos
    matches_table.write.jdbc(url=url, table="matches", mode="overwrite", properties=properties)
    general_stats_table.write.jdbc(url=url, table="general_stats", mode="overwrite", properties=properties)
    cards_table.write.jdbc(url=url, table="cards", mode="overwrite", properties=properties)
    corners_table.write.jdbc(url=url, table="corners", mode="overwrite", properties=properties)
    half_time_results_table.write.jdbc(url=url, table="half_time_results", mode="overwrite", properties=properties)
    betting_odds_table.write.jdbc(url=url, table="betting_odds", mode="overwrite", properties=properties)
    shots_on_target_table.write.jdbc(url=url, table="shots_on_target", mode="overwrite", properties=properties)
    substitutions_table.write.jdbc(url=url, table="substitutions", mode="overwrite", properties=properties)
    alternative_betting_odds_table.write.jdbc(url=url, table="alternative_betting_odds", mode="overwrite", properties=properties)

In [ ]:
#INSERCION DEL CSV
from pyspark.sql import SparkSession

# Sesión de Spark y Controlador PostgreSQL
spark = SparkSession.builder.appName("example").config("spark.driver.extraClassPath", "/content/postgresql-42.2.23.jar").getOrCreate()

# Conn de bd de render
host = "dpg-ck7g9u7q54js73famlu0-a.oregon-postgres.render.com"
port = 5432
database = "codema"
user = "codema_user"
password = "TL35jKeo53oDQkPDsw6c587xWAMSCmd6"

# Conexión
url = f"jdbc:postgresql://{host}:{port}/{database}"
properties = {"user": user, "password": password, "driver": "org.postgresql.Driver"}

try:
    # Leer el CSV y convertirlo en un DataFrame
    csv_file_path = "/content/premier1.csv"  # Reemplaza con la ruta correcta de tu archivo CSV
    premier_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

    # Muestra el DataFrame
    premier_df.show()

    # Escribir el DataFrame en la base de datos
    premier_df.write.jdbc(url=url, table="premier_league", mode="append", properties=properties)


except Exception as e:
    print("Error al leer o escribir datos:", str(e))
finally:
    # Cierra la sesión de Spark
    spark.stop()


+---+----------+-------------------+----------------+----------------+----+----+---+----+----+---+----------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+-----+----+-----+----+----+----+----+---+----+----+----+----+-----+----+-----+--------+--------+-----+-----+-------+-------+-------+-------+-----+-------+-------+----+----+------+------+------+------+------+------+------+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+---------+---------+------+------+--------+--------+--------+--------+-----+--------+--------+-----+-----+-------+-------+-------+-------+
|Div|      Date|               Time|        HomeTeam|        AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|   Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|B365H|B365D|B365A| BWH| BWD| BWA| IWH| IWD| IWA|  PSH| PSD|  PSA| WHH| WHD| WHA| VCH|VCD| VCA|MaxH|MaxD|MaxA| AvgH|AvgD| AvgA|B365>2.5|B365<2.5|P>2.5|P<2.5|Max>2.5|